In [1]:
%matplotlib notebook
import glob
import os
import numpy as np
import matplotlib.pyplot as plt
import tifffile as tif
import json

In [35]:
def read_tif_files():
    '''function to read out tif files in directory'''
    #usr_in = input('Please give path to where is your data located at:  ')
    f_list = list(glob.iglob('/Users/timothyliu/Desktop/Globus/multimodal/data3_Textured/**/*.tif', recursive=True))
    dark_list = list(glob.iglob('/Users/timothyliu/Desktop/Globus/multimodal/data3_Textured/**/*dark*.tif', recursive=True))
    for el in f_list:
        if el in dark_list:
            f_list.remove(el)
    #out_list = [f for f in f_list if f not in dark_list]
    print('number of total files = {}'.format(len(list(f_list))))
    print('number of dark files = {}'.format(len(list(dark_list))))
    return list(f_list)
# doesn't have to do this step, once we have f_list
f_list = read_tif_files()
#print(f_list)
with open('/Users/timothyliu/hackathon/texture_f_list.txt', 'w') as f:
    json.dump(f_list, f)

number of total files = 172
number of dark files = 20


In [3]:
# collect clean data
std_list = list(glob.glob('/Users/timothyliu/Desktop/Globus/multimodal/clean_data/*.tif'))
dark_list = []
for el in std_list:
    if 'dark' in el:
        dark_list.append(el)
        std_list.remove(el)
std_list

['/Users/timothyliu/Desktop/Globus/multimodal/clean_data/CeO2_calib-00000.tif',
 '/Users/timothyliu/Desktop/Globus/multimodal/clean_data/Ni_300K-00044.tif',
 '/Users/timothyliu/Desktop/Globus/multimodal/clean_data/Ni_STD_300K-00538.tif',
 '/Users/timothyliu/Desktop/Globus/multimodal/clean_data/nickel-00000.raw.tif',
 '/Users/timothyliu/Desktop/Globus/multimodal/clean_data/nickel_296K-00002.tif',
 '/Users/timothyliu/Desktop/Globus/multimodal/clean_data/nickelstandard_100K-00028.tif']

In [4]:
add_list = list(glob.glob('/Users/timothyliu/Desktop/Globus/multimodal/clean_data/**/*.tif'))
with open('/Users/timothyliu/hackathon/std_f_list.txt', 'w') as std_f:
    json.dump(std_list, std_f)
with open('/Users/timothyliu/hackathon/extra_std_f_list.txt', 'w') as add_f:
    json.dump(add_list, add_f)

In [5]:
# start to work
os.chdir('/Users/timothyliu/hackathon/')
# read in json file
with open('texture_f_list.txt','r') as f:
    f_list = json.load(f)
with open('untextured_f_list.txt','r') as f:
    std_list = json.load(f)

In [6]:
# brutally corp images
def quarter_img(f_name, center=1024, width=200):
    img = tif.imread(f_name)
    img_list = []
    for i in np.linspace(-1,1,2):
        for j in np.linspace(-1,1,2):
            lower_x = min(center+int(i)*width, center)
            upper_x = max(center+int(i)*width, center)
            lower_y = min(center+int(j)*width, center)
            upper_y = max(center+int(j)*width, center)
            img_list.append(img[lower_x:upper_x, lower_y:upper_y])
    return img_list

def crop_img(f_name, center=1024, width=200):
    img = tif.imread(f_name)
    img_list = []
    img_list.append(img[center-width:center+width, center-width:center+width])
    return img_list

In [39]:
# loop over images and create np array of texture data
def file_dict(f_list):
    out_dict = {}
    for i in range(len(f_list)):
        out_dict.update({str(i):os.path.basename(f_list[i])})
        #print(os.path.basename(f_list[i]))
    return out_dict
        
def build_quarter_img_list(f_list):
    import tifffile as tif
    out_list = []
    f_out_list = []
    _img = tif.imread(f_list[1])
    num_obs = len(f_list)
    (x_dim, y_dim) = np.shape(_img)
    for i in range(len(f_list)):
        quarter_imgs = quarter_img(f_list[i])
        for el in quarter_imgs:
            out_list.append(el)
            f_out_list.append(str(i))
    out = np.array(out_list)
    return out, f_out_list

def build_full_img_list(f_list):
    import tifffile as tif
    out_list = []
    _img = tif.imread(f_list[1])
    num_obs = len(f_list)
    (x_dim, y_dim) = np.shape(_img)
    for f in f_list:
        img = crop_img(f)
        out_list.append(img)
    _out = np.array(out_list)
    out = np.squeeze(_out)
    return out, f_list

In [40]:
# quarter_imgs
(quarter_texture_data_list, quarter_textrure_f_list) = build_quarter_img_list(f_list)
(quarter_standard_list, quarter_std_f_list) = build_quarter_img_list(std_list)
(quarter_extra_standard_list, quarter_extra_standard_f_list) = build_quarter_img_list(add_list)

# save data
os.makedirs('/Users/timothyliu/hackathon/quarter/', exist_ok=True)
np.save('/Users/timothyliu/hackathon/quarter/texture_data_np_array',quarter_texture_data_list)
np.save('/Users/timothyliu/hackathon/quarter/standard_data_np_array', quarter_standard_list)
np.save('/Users/timothyliu/hackathon/quarter/extra_standard_data_np_array', quarter_extra_standard_list)

"\nos.makedirs('/Users/timothyliu/hackathon/quarter/', exist_ok=True)\nnp.save('/Users/timothyliu/hackathon/quarter/texture_data_np_array',quarter_texture_data_list)\nnp.save('/Users/timothyliu/hackathon/quarter/standard_data_np_array', quarter_standard_list)\nnp.save('/Users/timothyliu/hackathon/quarter/extra_standard_data_np_array', quarter_extra_standard_list)\n"

In [42]:
len(quarter_textrure_f_list)

688

In [44]:
f_dict = file_dict(f_list)
f_dict

{'0': 'AuOnCu_bkg_RT-00000.tif',
 '1': 'AuOnCu_bkg_RT-00001.tif',
 '10': 'AuOnCu_bkg_RT-00011.tif',
 '100': 'Cu_in_polyester_225um_bkg_final-00010.tif',
 '101': 'Cu_in_polyester_225um_bkg_final-00011.tif',
 '102': 'Cu_in_polyester_225um_bkg_final-00012.tif',
 '103': 'Cu_in_polyester_225um_bkg_final-00013.tif',
 '104': 'Cu_in_polyester_225um_bkg_final-00014.tif',
 '105': 'Cu_in_polyester_225um_bkg_final-00015.tif',
 '106': 'Cu_in_polyester_225um_bkg_final-00016.tif',
 '107': 'Cu_in_polyester_225um_bkg_final-00017.tif',
 '108': 'Cu_in_polyester_225um_bkg_final-00018.tif',
 '109': 'Cu_in_polyester_225um_bkg_final-00019.tif',
 '11': 'AuOnCu_bkg_RT-00012.tif',
 '110': 'Cu_in_polyester_225um_bkg_final-00020.tif',
 '111': 'Cu_in_poly_test-00000.tif',
 '112': 'Cu_in_poly_test-00001.tif',
 '113': 'Cu_in_poly_test-00002.tif',
 '114': 'GLPF-1-1_RT-00001.tif',
 '115': 'GLPF-1-1_RT-00002.tif',
 '116': 'GLPF-1-1_RT-00003.tif',
 '117': 'GLPF-1-1_RT-00004.tif',
 '118': 'GLPF-1-1_RT-00005.tif',
 '119':

In [ ]:
# full imgs
texture_data_list = build_full_img_list(f_list)
standard_list = build_full_img_list(std_list)
extra_standard_list = build_full_img_list(add_list)

# save data
'''
os.makedirs('/Users/timothyliu/hackathon/full/', exist_ok=True)
np.save('/Users/timothyliu/hackathon/full/texture_data_np_array', texture_data_list)
np.save('/Users/timothyliu/hackathon/full/standard_data_np_array', standard_list)
np.save('/Users/timothyliu/hackathon/full/extra_standard_data_np_array', extra_standard_list)
'''

In [ ]:
# get file label
np.random.seed(1337)
